In [0]:
from google.colab import drive

In [0]:
drive.mount('/content/gdrive')

In [0]:
import torchvision
import torch
from torchvision import transforms


In [0]:
images = torchvision.datasets.ImageFolder("/content/gdrive/My Drive/blur_detection/TrainingSet/Undistorted", transform = transforms.TenCrop(64) )
raw_images = torchvision.datasets.ImageFolder("/content/gdrive/My Drive/blur_detection/TrainingSet/Undistorted", transform = transforms.TenCrop(64))
org_blurred = torchvision.datasets.ImageFolder("/content/gdrive/My Drive/blur_detection/TrainingSet/Naturally-Blurred", transform = transforms.FiveCrop(64))
art_blurred = torchvision.datasets.ImageFolder("/content/gdrive/My Drive/blur_detection/TrainingSet/Artificially-Blurred", transform = transforms.FiveCrop(64))

In [0]:
# X_train = [x for (x, y) in images]
# Y_target = [1 if x == 0 or x == 1 else 0 for (x, y) in images]
from torch.utils.data import random_split
from PIL import ImageFilter
#0 - sharp
#1 - blurred

dataset = []
default_transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5 , 0.5, 0.5), (0.5, 0.5, 0.5))])

for (x, y) in images:
  for tnsr in x:
    dataset.append([default_transforms(tnsr), 0])

for (x, y) in raw_images:
  for tnsr in x:
    img = tnsr.filter(ImageFilter.BLUR).copy()
    dataset.append([default_transforms(img), 1])

for (x, y) in org_blurred:
  for tnsr in x:
    dataset.append([default_transforms(tnsr), 1])


for (x, y) in art_blurred:
  for tnsr in x:
    dataset.append([default_transforms(tnsr), 1])


train_dataset, test_dataset = random_split(dataset, [int(len(dataset) * 0.8), int(len(dataset) * 0.2)])

In [0]:
print(images[0])
print(dataset[4])

In [0]:
import torch.utils.data as data
train_loader = data.DataLoader(dataset = train_dataset, batch_size = 50, shuffle = True)
test_lodaer = data.DataLoader(dataset = test_dataset, batch_size = 100, shuffle = True)

In [0]:
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class BlurDetector(nn.Module):
  def __init__(self):
    super(BlurDetector, self).__init__()
    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 12, kernel_size = 7, stride = 1, padding = 1)
    self.relu1 = nn.LeakyReLU()
    self.maxpool1 = nn.MaxPool2d(kernel_size = 2)
    self.dropout1 = nn.Dropout(0.2)
  
    self.conv2 = nn.Conv2d(in_channels = 12, out_channels = 36, kernel_size = 5, stride = 1, padding = 1)
    self.relu2 = nn.LeakyReLU()
    self.maxpool2 = nn.MaxPool2d(kernel_size = 2)
    self.dropout2 = nn.Dropout(0.2)

    self.conv3 = nn.Conv2d(in_channels = 36, out_channels = 108, kernel_size = 3, stride = 1, padding = 1)
    self.relu3 = nn.LeakyReLU()
    self.maxpool3 = nn.MaxPool2d(kernel_size = 2)
    self.dropout3 = nn.Dropout(0.2)

    self.relu4 = nn.LeakyReLU()

    self.sigmoid = nn.Sigmoid()

    # self.lf = nn.Linear(in_features = 7 * 7 * 108, out_features = 2)
    self.lf = nn.Linear(in_features = 14 * 14 * 36, out_features = 2)
    self.softmax = nn.Softmax()

  def forward(self, input):

    output = self.conv1(input)
    output = self.relu1(output)
    output = self.maxpool1(output)
    output = self.dropout1(output)

    output = self.conv2(output)
    output = self.relu2(output)
    output = self.maxpool2(output)
    output = self.dropout2(output)

    # output = self.conv3(output)
    # output = self.relu3(output)
    # output = self.maxpool3(output)
    #output = self.dropout3(output)
    
    output = self.relu4(output)
    # output = output.view(-1, 7 * 7 * 108)
    output = output.view(-1, 14 * 14 * 36)

    output = self.lf(output)
    
    #output = self.softmax(output)
    #output = self.sigmoid(output)

    return output

  def pokazTensor(self, input):
    print(input)

blurDetector = BlurDetector()


In [0]:
import torch.optim as optim

loss_fun = nn.CrossEntropyLoss()
optimizer = optim.Adam(blurDetector.parameters())
#optimizer = optim.SGD(blurDetector.parameters(), lr = 0.1, momentum = 0.9 )
##optimizer = optim.ASGD(blurDetector.parameters())

In [0]:

counter = 0
for epoch in range(9):
  print('Epoch ' + str(epoch))

  blurDetector.train()

  for i, (imgs, labels) in enumerate(train_loader):

    optimizer.zero_grad()
    outputs = blurDetector(imgs)
    loss = loss_fun(outputs, labels)
    loss.backward()
    optimizer.step()


    counter += 1
    if(not (counter % 100)):
      test_acc_count = 0
      for k, (test_imgs, test_labels) in enumerate(train_loader):
        test_outputs = blurDetector(test_imgs)
        _, prediction = torch.max(test_outputs.data, 1)
        test_acc_count += torch.sum(prediction == test_labels.data).item()

      test_accuracy = test_acc_count / len(train_dataset)
      print("Train data: Accuracy", test_accuracy)
      print("Train data: Loss", loss.item())

      test_acc_count = 0
      for k, (test_imgs, test_labels) in enumerate(test_lodaer):
        test_outputs = blurDetector(test_imgs)
        _, prediction = torch.max(test_outputs.data, 1)
        test_acc_count += torch.sum(prediction == test_labels.data).item()

      test_accuracy = test_acc_count / len(test_dataset)
      print("Test data: Accuracy", test_accuracy)
      print("Test data: Loss", loss.item())





print('Finished Training')

Epoch 0
Train data: Accuracy 0.5759515570934256
Train data: Loss 0.6771106123924255
Test data: Accuracy 0.5788927335640138
Test data: Loss 0.6771106123924255
Train data: Accuracy 0.5598615916955018
Train data: Loss 0.711914598941803
Test data: Accuracy 0.5709342560553633
Test data: Loss 0.711914598941803
Epoch 1
Train data: Accuracy 0.6136678200692042
Train data: Loss 0.6126881837844849
Test data: Accuracy 0.6186851211072665
Test data: Loss 0.6126881837844849
Train data: Accuracy 0.6161764705882353
Train data: Loss 0.608364462852478
Test data: Accuracy 0.6131487889273356
Test data: Loss 0.608364462852478
Epoch 2
Train data: Accuracy 0.6417820069204152
Train data: Loss 0.6135594248771667
Test data: Accuracy 0.6370242214532872
Test data: Loss 0.6135594248771667
Train data: Accuracy 0.5980968858131488
Train data: Loss 0.6358479857444763
Test data: Accuracy 0.5955017301038062
Test data: Loss 0.6358479857444763
Epoch 3
Train data: Accuracy 0.6671280276816609
Train data: Loss 0.5886031985282

In [0]:
blurDetector.eval()
test_acc_count = 0
for k, (test_imgs, test_labels) in enumerate(test_lodaer):
  #print(test_imgs)
  test_outputs = blurDetector(test_imgs)
  _, prediction = torch.max(test_outputs.data, 1)
  #print(torch.max(test_outputs.data, 1))
  test_acc_count += torch.sum(prediction == test_labels.data).item()

test_accuracy = test_acc_count / len(test_dataset)
test_accuracy


0.683044982698962

In [0]:
print("Manual testing")

Manual testing


In [0]:

detector = torch.jit.load("/content/gdrive/My Drive/blur_detection/blurDetector.pt")
detector.eval()

ScriptModule(
  original_name=BlurDetector
  (conv1): ScriptModule(original_name=Conv2d)
  (relu1): ScriptModule(original_name=LeakyReLU)
  (maxpool1): ScriptModule(original_name=MaxPool2d)
  (conv2): ScriptModule(original_name=Conv2d)
  (relu2): ScriptModule(original_name=LeakyReLU)
  (maxpool2): ScriptModule(original_name=MaxPool2d)
  (conv3): ScriptModule(original_name=Conv2d)
  (relu3): ScriptModule(original_name=LeakyReLU)
  (maxpool3): ScriptModule(original_name=MaxPool2d)
  (relu4): ScriptModule(original_name=LeakyReLU)
  (lf): ScriptModule(original_name=Linear)
)

In [0]:
from PIL import Image
testFullSizes = []
testFullSizes.append( Image.open("/content/gdrive/My Drive/blur_detection/blurred.jpg") )
testFullSizes.append( Image.open("/content/gdrive/My Drive/blur_detection/blurred_max.jpg") )
testFullSizes.append( Image.open("/content/gdrive/My Drive/blur_detection/test.jpg") )
testFullSizes.append( Image.open("/content/gdrive/My Drive/blur_detection/hq2.jpg") )
testFullSizes.append( Image.open("/content/gdrive/My Drive/blur_detection/sharp.jpg") )
testFullSizes.append( Image.open("/content/gdrive/My Drive/blur_detection/treningowe.jpg") )
testFullSizes.append( Image.open("/content/gdrive/My Drive/blur_detection/treningowe2.jpg") )
testFullSizes.append( Image.open("/content/gdrive/My Drive/blur_detection/treningowe3.jpg") )
testFullSizes.append( Image.open("/content/gdrive/My Drive/blur_detection/pies.jpg") )
testFullSizes.append( Image.open("/content/gdrive/My Drive/blur_detection/small_sharp.jpg") )


slicedTensors = []
trs = transforms.Compose ( [transforms.ToTensor(), transforms.Normalize((0.5 , 0.5, 0.5), (0.5, 0.5, 0.5))] )

for idx, img in enumerate(testFullSizes):
  imgwidth, imgheight = img.size
  img = img.resize( (imgwidth - imgwidth % 64, imgheight - imgheight % 64))
  slicedTensors.append([])

  imgwidth, imgheight = img.size
  for i in range(0, imgheight, 64):
    for j in range(0, imgwidth, 64):
      box = (j, i, j + 64, i + 64)
      cropped = img.crop(box).copy()
      slicedTensors[idx].append(trs(cropped))





In [0]:


for idx, bigSlc in enumerate(slicedTensors):
  result = []

  #print(len(bigSlc))
  for slc in bigSlc:
    one_img = slc
    one_img = one_img[None]
    one_img = one_img.type('torch.FloatTensor')
    output_img = blurDetector(one_img)
    #print(output_img)
    _, prediction = torch.max(output_img.data, 1)
    result.extend(prediction.tolist())
  
  print("Index of image: ", idx)
  print("Sharp segments: ", result.count(0))
  print("Blurred segments: ", result.count(1))
  print("")

Index of image:  0
Sharp segments:  58
Blurred segments:  452

Index of image:  1
Sharp segments:  0
Blurred segments:  91

Index of image:  2
Sharp segments:  60
Blurred segments:  196

Index of image:  3
Sharp segments:  135
Blurred segments:  57

Index of image:  4
Sharp segments:  30
Blurred segments:  5

Index of image:  5
Sharp segments:  159
Blurred segments:  813

Index of image:  6
Sharp segments:  467
Blurred segments:  1261

Index of image:  7
Sharp segments:  10
Blurred segments:  1718

Index of image:  8
Sharp segments:  3
Blurred segments:  1935

Index of image:  9
Sharp segments:  17
Blurred segments:  133



In [0]:
slc = slicedTensors[1][0]
one_img = slc
one_img = one_img[None]
one_img = one_img.type('torch.FloatTensor')
print(one_img)

tensor([[[[ 0.2157,  0.2157,  0.2078,  ..., -0.2314, -0.2549, -0.2549],
          [ 0.2078,  0.2078,  0.2000,  ..., -0.2392, -0.2627, -0.2627],
          [ 0.2000,  0.1922,  0.1922,  ..., -0.2471, -0.2706, -0.2706],
          ...,
          [ 0.0039,  0.0039, -0.0039,  ..., -0.1373, -0.1765, -0.1765],
          [ 0.0039, -0.0039, -0.0039,  ..., -0.1294, -0.1686, -0.1686],
          [-0.0118, -0.0118, -0.0118,  ..., -0.1216, -0.1451, -0.1451]],

         [[ 0.2549,  0.2549,  0.2471,  ..., -0.2000, -0.1922, -0.1922],
          [ 0.2471,  0.2471,  0.2392,  ..., -0.2078, -0.2000, -0.2000],
          [ 0.2392,  0.2314,  0.2314,  ..., -0.2157, -0.2078, -0.2078],
          ...,
          [ 0.0510,  0.0510,  0.0431,  ..., -0.1137, -0.1373, -0.1373],
          [ 0.0510,  0.0431,  0.0431,  ..., -0.1059, -0.1294, -0.1294],
          [ 0.0353,  0.0353,  0.0353,  ..., -0.0980, -0.0980, -0.0980]],

         [[-0.2000, -0.2000, -0.2078,  ..., -0.5922, -0.5608, -0.5608],
          [-0.2078, -0.2078, -

In [0]:
testFullSizes[0].size

(1920, 1090)

In [0]:
tt = trs =  transforms.ToTensor()
test_without_grayscale = Image.open("/content/gdrive/My Drive/blur_detection/blurred_max.jpg")
print(trs(test_without_grayscale)[0][0][0].type())



torch.FloatTensor


In [0]:
# torch.jit.save(blurDetector, "/content/gdrive/My Drive/blur_detection/blurDetector.pt")
blurDetector.eval()
script = torch.jit.script(blurDetector)
script.save("/content/gdrive/My Drive/blur_detection/blurDetector6.pt")